In [ ]:
!pip install gensim --quiet
!pip install tensorflow-datasets --quiet
!pip install tensorflow-text==2.15.0 --quiet #15
!pip install tensorflow==2.15.0 --quiet #15 13
!pip install tf_keras==2.15.0 --quiet
!pip install transformers==4.17 --quiet
!pip install pydot --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 26.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import tensorflow_datasets as tfds
import tensorflow_text as tf_text
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


import sklearn as sk
import os
import nltk
from nltk.data import find

import matplotlib.pyplot as plt

import re


import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.test.utils import datapath

In [ ]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

Mounted at /content/drive


# Data Prep

In [ ]:
wiki_data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/wiki_movie_plots_deduped.csv")

In [ ]:

import glob
import pandas as pd

# get data file names
path =r'/content/drive/My Drive/Colab Notebooks/movie genres'
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

genre_dataset = pd.concat(li, axis=0, ignore_index=True)
genre_dataset.drop_duplicates(subset="movie_name", keep='first', inplace=True)


In [ ]:
# prompt: join data and frame on frame.movie_name and data.title

movie_genre_data = pd.merge(genre_dataset, wiki_data, left_on='movie_name', right_on='Title')
movie_genre_data = movie_genre_data.drop('Genre', axis=1)
movie_genre_data['top_genre'] = movie_genre_data['genre'].str.split(',').str[0]
movie_genre_data = movie_genre_data[~movie_genre_data['top_genre'].isin(['Sport', 'Western', 'Adult', 'Reality-TV'])]

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
movie_genre_data['genre_encoded'] = label_encoder.fit_transform(movie_genre_data['top_genre'])

data = movie_genre_data


<ipython-input-6-0bafb3e702b2>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_genre_data['genre_encoded'] = label_encoder.fit_transform(movie_genre_data['top_genre'])


In [ ]:
genre_counts = movie_genre_data['top_genre'].value_counts().to_dict()
print(genre_counts)

{'Action': 6821, 'Comedy': 4247, 'Drama': 3634, 'Crime': 2423, 'Adventure': 1948, 'Horror': 927, 'Biography': 778, 'Animation': 654, 'Romance': 380, 'Thriller': 216, 'Mystery': 205, 'Fantasy': 175, 'Family': 168, 'Musical': 95, 'Sci-Fi': 61, 'History': 33, 'Film-Noir': 21, 'Music': 21, 'War': 18}


In [ ]:
nltk.download('word2vec_sample')


[nltk_data] Downloading package word2vec_sample to /root/nltk_data...
[nltk_data]   Unzipping models/word2vec_sample.zip.


True

In [ ]:
word2vec_sample = str(find('models/word2vec_sample/pruned.word2vec.txt'))
vec_model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_sample, binary=False)

In [ ]:
EMBEDDING_DIM = 300

# initialize embedding matrix and word-to-id map:
embedding_matrix = np.zeros((len(vec_model) + 1, EMBEDDING_DIM))
vocab_dict = {}

# build the embedding matrix and the word-to-id map:
for i, word in enumerate(vec_model.index_to_key):
    embedding_vector = vec_model[word]

    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        vocab_dict[word] = i

# we can use the last index at the end of the vocab for unknown tokens
vocab_dict['[UNK]'] = len(vocab_dict)

In [ ]:
X = data['Plot'].values  # Features (examples)
y = data['genre_encoded'].values  # Labels

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert the data to TensorFlow datasets
train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_data = tf.data.Dataset.from_tensor_slices((X_test, y_test))


In [ ]:
X_train[0]

'In the prologue, which is set in medieval times, the audience is introduced to humans with special powers called The Others (Иные, иной; Inyye, Inoy). The Others are divided into two camps, with each Other either allied with the forces of Light or the forces of Darkness, and the prologue recounts a great battle between the two sides. Geser, the Lord of the Light, realized that the two forces would eventually annihilate each other. He stopped the battle and offered a truce to the forces of Darkness, led by Zavulon. To make sure each side honored its side of the agreement, the Light would have a "Night Watch" to monitor the forces of Darkness at night, and the Darkness would have a "Day Watch" to police the forces of Light during the day. This agreement maintained balance for centuries to come.\r\nIn modern-day Moscow, Anton Gorodetsky (Russian: Антон Городецкий) visits a witch named Daria and asks her to cast a spell to return his wife to him, agreeing that she should miscarry her chil

In [ ]:
train_examples, train_labels = next(iter(train_data.batch(20000)))
test_examples, test_labels = next(iter(test_data.batch(5000)))


In [ ]:
tokenizer = tf_text.WhitespaceTokenizer()
train_tokens = tokenizer.tokenize(train_examples)
test_tokens = tokenizer.tokenize(test_examples)

In [ ]:
MAX_SEQUENCE_LENGTH = 100
NUM_CLASSES = len(data.top_genre.unique())

In [ ]:
NUM_CLASSES

19

In [ ]:
def docs_to_vocab_ids(tokenized_texts_list, max_sequence_length = MAX_SEQUENCE_LENGTH):
    """
    converting a list of strings to a list of lists of word ids
    """
    texts_vocab_ids = []
    for i, token_list in enumerate(tokenized_texts_list):

        # Get the vocab id for each token in this doc ([UNK] if not in vocab)
        vocab_ids = []
        for token in list(token_list.numpy()):
            decoded = token.decode('utf-8', errors='ignore')
            if decoded in vocab_dict:
                vocab_ids.append(vocab_dict[decoded])
            else:
                vocab_ids.append(vocab_dict['[UNK]'])

        # Truncate text to max length, add padding up to max length
        vocab_ids = vocab_ids[:max_sequence_length]
        n_padding = (max_sequence_length - len(vocab_ids))
        # For simplicity in this model, we'll just pad with uknown tokens
        vocab_ids += [vocab_dict['[UNK]']] * n_padding

        # Add this example to the list of converted docs
        texts_vocab_ids.append(vocab_ids)

        if i % 5000 == 0:
            print('Examples processed: ', i)

    print('Total examples: ', i)

    return np.array(texts_vocab_ids)

In [ ]:
train_input = docs_to_vocab_ids(train_tokens)
test_input = docs_to_vocab_ids(test_tokens)

train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

Examples processed:  0
Examples processed:  5000
Examples processed:  10000
Examples processed:  15000
Total examples:  18259
Examples processed:  0
Total examples:  4564


# Modeling

## Averaging network

In [ ]:
def build_an_model(MAX_SEQUENCE_LENGTH = MAX_SEQUENCE_LENGTH):
    an_input_layer = tf.keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int64')
    an_embedding_layer = Embedding(embedding_matrix.shape[0],
                               embedding_matrix.shape[1],
                               embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
                               input_length=MAX_SEQUENCE_LENGTH,
                               trainable=False)
    an_embeddings = an_embedding_layer(an_input_layer)
    an_avg_embeddings = tf.keras.layers.Lambda(lambda x: K.mean(x, axis=1), name='averaging')(an_embeddings)
    an_classification = tf.keras.layers.Dense(NUM_CLASSES,
                                          activation='sigmoid',
                                          name='an_classification')(an_avg_embeddings)
    an_model = tf.keras.models.Model(inputs=an_input_layer, outputs=[an_classification])
    an_model.compile(loss='sparse_categorical_crossentropy',
                 optimizer=tf.keras.optimizers.Adam(learning_rate=0.001,
                                                    beta_1=0.9,
                                                    beta_2=0.999,
                                                    epsilon=1e-07,
                                                    amsgrad=False,
                                                    name='Adam'),
                 metrics=['accuracy'])
    return an_model


In [ ]:
an_model = build_an_model()
an_history = an_model.fit(train_input,
                          train_labels,
                          validation_data=(test_input, test_labels),
                          batch_size=32,
                          epochs=20
                          )

Epoch 1/20
571/571 [==============================] - 3s 3ms/step - loss: 2.4375 - accuracy: 0.2885 - val_loss: 2.1964 - val_accuracy: 0.3012
Epoch 2/20
571/571 [==============================] - 2s 3ms/step - loss: 2.1566 - accuracy: 0.2982 - val_loss: 2.1046 - val_accuracy: 0.3012
Epoch 3/20
571/571 [==============================] - 2s 3ms/step - loss: 2.0992 - accuracy: 0.2982 - val_loss: 2.0654 - val_accuracy: 0.3012
Epoch 4/20
571/571 [==============================] - 2s 3ms/step - loss: 2.0669 - accuracy: 0.2990 - val_loss: 2.0391 - val_accuracy: 0.3027
Epoch 5/20
571/571 [==============================] - 2s 4ms/step - loss: 2.0437 - accuracy: 0.3029 - val_loss: 2.0196 - val_accuracy: 0.3106
Epoch 6/20
571/571 [==============================] - 2s 3ms/step - loss: 2.0260 - accuracy: 0.3101 - val_loss: 2.0041 - val_accuracy: 0.3205
Epoch 7/20
571/571 [==============================] - 2s 3ms/step - loss: 2.0118 - accuracy: 0.3165 - val_loss: 1.9917 - val_accuracy: 0.3244
Epoch 

In [ ]:
train_input_100l = docs_to_vocab_ids(train_tokens, 100)
test_input_100l = docs_to_vocab_ids(test_tokens, 100)

an_model_2 = build_an_model(100)
an_history = an_model_2.fit(train_input_100l,
                          train_labels,
                          validation_data=(test_input_100l, test_labels),
                          batch_size=32,
                          epochs=20
                          )

Examples processed:  0
Examples processed:  5000
Examples processed:  10000
Examples processed:  15000
Total examples:  18259
Examples processed:  0
Total examples:  4564
Epoch 1/20
571/571 [==============================] - 2s 3ms/step - loss: 2.3522 - accuracy: 0.2880 - val_loss: 2.1075 - val_accuracy: 0.3012
Epoch 2/20
571/571 [==============================] - 2s 3ms/step - loss: 2.0880 - accuracy: 0.2982 - val_loss: 2.0483 - val_accuracy: 0.3012
Epoch 3/20
571/571 [==============================] - 2s 3ms/step - loss: 2.0536 - accuracy: 0.2984 - val_loss: 2.0260 - val_accuracy: 0.3025
Epoch 4/20
571/571 [==============================] - 2s 3ms/step - loss: 2.0345 - accuracy: 0.3035 - val_loss: 2.0102 - val_accuracy: 0.3117
Epoch 5/20
571/571 [==============================] - 2s 3ms/step - loss: 2.0193 - accuracy: 0.3123 - val_loss: 1.9976 - val_accuracy: 0.3146
Epoch 6/20
571/571 [==============================] - 2s 4ms/step - loss: 2.0065 - accuracy: 0.3186 - val_loss: 1.9863 

## Deep Averaging Network

In [ ]:
def build_dan_model(hidden_layer_dims, dropout_rate, embeddings_trainable=False, MAX_SEQUENCE_LENGTH = MAX_SEQUENCE_LENGTH):
    dan_input_layer = tf.keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int64')
    dan_embedding_layer = Embedding(embedding_matrix.shape[0],
                                    embedding_matrix.shape[1],
                                    embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
                                    input_length=MAX_SEQUENCE_LENGTH,
                                    trainable=embeddings_trainable)
    dan_embeddings = dan_embedding_layer(dan_input_layer)
    dan_avg_embeddings = tf.keras.layers.Lambda(lambda x: K.mean(x, axis=1),
                                                name='averaging')(dan_embeddings)
    last_hidden_output = dan_avg_embeddings

    for i, h_dim in enumerate(hidden_layer_dims):
        last_hidden_output = tf.keras.layers.Dense(h_dim, activation='relu',
                                                   name='dan_hidden_%d' % i)(last_hidden_output)
        last_hidden_output = tf.keras.layers.Dropout(dropout_rate)(last_hidden_output)

    dan_classification = tf.keras.layers.Dense(NUM_CLASSES,
                                               activation='sigmoid',
                                               name='dan_classification')(last_hidden_output)

    dan_model = tf.keras.models.Model(inputs=dan_input_layer, outputs=[dan_classification])
    dan_model.compile(loss='sparse_categorical_crossentropy',
                      optimizer=tf.keras.optimizers.Adam(learning_rate=0.001,
                                                         beta_1=0.9,
                                                         beta_2=0.999,
                                                         epsilon=1e-07,
                                                         amsgrad=False,
                                                         name='Adam'),
                      metrics=['accuracy'])
    return dan_model

In [ ]:
hidden_layer_sizes = [100, 50]
dropout_rate = 0.5


In [ ]:
dan_model = build_dan_model(hidden_layer_sizes, dropout_rate)
dan_history = dan_model.fit(train_input,
                            train_labels,
                            validation_data=(test_input, test_labels),
                            batch_size=32,
                            epochs=20
                            )

Epoch 1/20
571/571 [==============================] - 5s 4ms/step - loss: 2.2074 - accuracy: 0.2750 - val_loss: 2.0316 - val_accuracy: 0.3012
Epoch 2/20
571/571 [==============================] - 2s 4ms/step - loss: 2.0613 - accuracy: 0.3085 - val_loss: 1.9842 - val_accuracy: 0.3459
Epoch 3/20
571/571 [==============================] - 3s 5ms/step - loss: 2.0087 - accuracy: 0.3381 - val_loss: 1.9413 - val_accuracy: 0.3619
Epoch 4/20
571/571 [==============================] - 3s 5ms/step - loss: 1.9722 - accuracy: 0.3516 - val_loss: 1.9129 - val_accuracy: 0.3728
Epoch 5/20
571/571 [==============================] - 2s 4ms/step - loss: 1.9465 - accuracy: 0.3619 - val_loss: 1.9020 - val_accuracy: 0.3717
Epoch 6/20
571/571 [==============================] - 2s 4ms/step - loss: 1.9266 - accuracy: 0.3662 - val_loss: 1.8749 - val_accuracy: 0.3766
Epoch 7/20
571/571 [==============================] - 2s 4ms/step - loss: 1.9132 - accuracy: 0.3681 - val_loss: 1.8606 - val_accuracy: 0.3801
Epoch 

In [ ]:
dan_model_100l = build_dan_model(hidden_layer_sizes, dropout_rate, MAX_SEQUENCE_LENGTH = 100)
dan_history_ = dan_model_100l.fit(train_input_100l,
                            train_labels,
                            validation_data=(test_input_100l, test_labels),
                            batch_size=32,
                            epochs=20
                            )

Epoch 1/20
571/571 [==============================] - 4s 5ms/step - loss: 2.1843 - accuracy: 0.2826 - val_loss: 2.0195 - val_accuracy: 0.3012
Epoch 2/20
571/571 [==============================] - 2s 3ms/step - loss: 2.0505 - accuracy: 0.3171 - val_loss: 1.9740 - val_accuracy: 0.3525
Epoch 3/20
571/571 [==============================] - 2s 4ms/step - loss: 2.0063 - accuracy: 0.3370 - val_loss: 1.9448 - val_accuracy: 0.3619
Epoch 4/20
571/571 [==============================] - 2s 3ms/step - loss: 1.9824 - accuracy: 0.3483 - val_loss: 1.9353 - val_accuracy: 0.3643
Epoch 5/20
571/571 [==============================] - 2s 3ms/step - loss: 1.9608 - accuracy: 0.3546 - val_loss: 1.9181 - val_accuracy: 0.3650
Epoch 6/20
571/571 [==============================] - 3s 5ms/step - loss: 1.9465 - accuracy: 0.3588 - val_loss: 1.9020 - val_accuracy: 0.3744
Epoch 7/20
571/571 [==============================] - 3s 5ms/step - loss: 1.9382 - accuracy: 0.3601 - val_loss: 1.8964 - val_accuracy: 0.3709
Epoch 

## CNN Model

In [ ]:
def build_cnn_model(num_filters, kernel_sizes, dense_layer_dims, dropout_rate, embeddings_trainable=False):
    cnn_input_layer = tf.keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int64')
    cnn_embedding_layer = Embedding(embedding_matrix.shape[0],
                                    embedding_matrix.shape[1],
                                    embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
                                    input_length=MAX_SEQUENCE_LENGTH,
                                    trainable=embeddings_trainable)
    cnn_embeddings = cnn_embedding_layer(cnn_input_layer)

    conv_layers_for_all_kernel_sizes = []
    for kernel_size, filters in zip(kernel_sizes, num_filters):
        conv_layer = keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu')(cnn_embeddings)
        conv_layer = keras.layers.GlobalMaxPooling1D()(conv_layer)
        conv_layers_for_all_kernel_sizes.append(conv_layer)

    conv_output = keras.layers.concatenate(conv_layers_for_all_kernel_sizes, axis=1)
    last_hidden_output = keras.layers.Dropout(rate=dropout_rate)(conv_output)

    for dense_layer_dim in dense_layer_dims:
        last_hidden_output = keras.layers.Dense(dense_layer_dim, activation='relu')(last_hidden_output)

    cnn_prediction = keras.layers.Dense(NUM_CLASSES, activation='sigmoid')(last_hidden_output)

    cnn_model = keras.Model(inputs=cnn_input_layer, outputs=cnn_prediction)
    cnn_model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',  # From information theory notebooks.
                      metrics=['accuracy'])
    return cnn_model


In [ ]:
num_filters = [100, 100, 50, 25]
kernel_sizes = [3, 4, 5, 10]
dense_layer_dims = [100, 50]
dropout_rate = 0.5

cnn_model = build_cnn_model(num_filters, kernel_sizes, dense_layer_dims, dropout_rate)

In [ ]:
cnn_history = cnn_model.fit(train_input,
                            train_labels,
                            validation_data=(test_input, test_labels),
                            batch_size=32,
                            epochs=5
                            )

Epoch 1/5
571/571 [==============================] - 13s 16ms/step - loss: 1.9952 - accuracy: 0.3415 - val_loss: 1.8416 - val_accuracy: 0.3755
Epoch 2/5
571/571 [==============================] - 9s 15ms/step - loss: 1.8128 - accuracy: 0.3982 - val_loss: 1.7429 - val_accuracy: 0.4228
Epoch 3/5
571/571 [==============================] - 8s 15ms/step - loss: 1.6997 - accuracy: 0.4350 - val_loss: 1.6857 - val_accuracy: 0.4370
Epoch 4/5
571/571 [==============================] - 9s 15ms/step - loss: 1.6068 - accuracy: 0.4665 - val_loss: 1.6819 - val_accuracy: 0.4311
Epoch 5/5
571/571 [==============================] - 8s 15ms/step - loss: 1.5149 - accuracy: 0.4983 - val_loss: 1.6592 - val_accuracy: 0.4471


## BERT MODEL

In [ ]:
from transformers import BertTokenizer, TFBertModel

checkpoint = 'bert-base-cased'
bert_tokenizer = BertTokenizer.from_pretrained(checkpoint)
bert_model = TFBertModel.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
def create_bert_multiclass_model(checkpoint = checkpoint,
                                 num_classes = NUM_CLASSES,
                                 hidden_size = 768,
                                 dropout=0.3,
                                 learning_rate=0.00005):
    """
    Build a simple classification model with BERT. Use the Pooler Output for classification purposes.
    """
    ### YOUR CODE HERE

    for layer in bert_model.layers:
        layer.trainable = True

    input_ids = tf.keras.layers.Input(shape=(None,), dtype=tf.int64, name='input_ids_layer')
    token_type_ids = tf.keras.layers.Input(shape=(None,), dtype=tf.int64, name='token_type_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(None,), dtype=tf.int64, name='attention_mask_layer')

    bert_inputs = {'input_ids': input_ids,
                   'token_type_ids': token_type_ids,
                   'attention_mask': attention_mask}

    bert_out = bert_model(bert_inputs)

    pooler_token = bert_out[1]
    #cls_token = bert_out[0][:, 0, :]

    hidden = tf.keras.layers.Dense(hidden_size, activation='relu', name='hidden_layer')(pooler_token)


    hidden = tf.keras.layers.Dropout(dropout)(hidden)


    classification = tf.keras.layers.Dense(num_classes, activation='softmax',name='classification_layer')(hidden)

    classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=[classification])

    classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                                 loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                                 metrics='accuracy')


    ### END YOUR CODE
    return classification_model

In [ ]:
def create_encodings_and_get_bert_model_with_max_length(max_length):
  train_encodings = bert_tokenizer(train_texts_bert, truncation=True, padding=True, max_length=max_length, return_tensors='tf')
  valid_encodings = bert_tokenizer(valid_texts_bert, truncation=True, padding=True, max_length=max_length, return_tensors='tf')
  test_encodings = bert_tokenizer(test_texts_bert, truncation=True, padding=True, max_length=max_length, return_tensors='tf')
  model = create_bert_multiclass_model()

  return train_encodings, valid_encodings, test_encodings, model


In [ ]:
train_texts_bert = X_train.tolist()
test_texts_bert = X_test.tolist()
valid_texts_bert = train_texts_bert[16000:]
valid_labels_bert = y_train[16000:]
train_texts_bert = train_texts_bert[:16000]
train_labels_bert = y_train[:16000]
test_labels_bert = y_test
npvalid_labels = np.asarray(valid_labels_bert)

In [ ]:
target_names = data[['top_genre', 'genre_encoded']].drop_duplicates().sort_values('genre_encoded')['top_genre'].tolist()


In [ ]:
train_encodings, valid_encodings, test_encodings, pooler_bert_model = create_encodings_and_get_bert_model_with_max_length(100)
pooler_bert_model_history = pooler_bert_model.fit([train_encodings.input_ids, train_encodings.token_type_ids, train_encodings.attention_mask],
                                                  train_labels,
                                                  validation_data=([valid_encodings.input_ids, valid_encodings.token_type_ids, valid_encodings.attention_mask],
                                                  npvalid_labels),
                                                  batch_size=8,
                                                  epochs=3)

Epoch 1/3
2000/2000 [==============================] - 409s 187ms/step - loss: 1.1417 - accuracy: 0.7163 - val_loss: 1.3052 - val_accuracy: 0.6580
Epoch 2/3
2000/2000 [==============================] - 368s 184ms/step - loss: 0.8412 - accuracy: 0.7930 - val_loss: 1.4051 - val_accuracy: 0.6350
Epoch 3/3
2000/2000 [==============================] - 368s 184ms/step - loss: 0.6541 - accuracy: 0.8409 - val_loss: 1.4886 - val_accuracy: 0.6296


In [ ]:
from sklearn.metrics import classification_report
predictions = pooler_bert_model.predict([test_encodings.input_ids, test_encodings.token_type_ids, test_encodings.attention_mask])

#now we need to get the highest probability in the distribution for each prediction
#and store that in a tf.Tensor
predictions_model1 = tf.argmax(predictions, axis=-1)
print(classification_report(test_labels, predictions_model1.numpy(), target_names=target_names))


143/143 [==============================] - 35s 210ms/step
              precision    recall  f1-score   support

      Action       0.71      0.75      0.73      1375
   Adventure       0.47      0.30      0.37       408
   Animation       0.67      0.46      0.54       113
   Biography       0.79      0.50      0.61       155
      Comedy       0.65      0.81      0.72       877
       Crime       0.71      0.63      0.66       493
       Drama       0.61      0.59      0.60       694
      Family       0.00      0.00      0.00        32
     Fantasy       0.50      0.03      0.05        36
   Film-Noir       0.00      0.00      0.00         6
     History       0.00      0.00      0.00         7
      Horror       0.44      0.75      0.56       182
       Music       0.00      0.00      0.00         4
     Musical       0.00      0.00      0.00        15
     Mystery       0.07      0.10      0.09        39
     Romance       0.21      0.15      0.18        73
      Sci-Fi       0.21

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
